# BATS climatologies 

The objective of the present notebook is to display the climatologies generated for the BATS station in the frame of the SEAMLESS project for the purpose of initialisation and overview of local dynamics. 

The steps involves : 
1. Loading the files generated in gotm-friendly formats. 
2. Integrate the data in an xarray structure
3. Make some plots
4. Output netcdf equivalent to these files, for any type of later usage. 

We assume the climatology files have been produced using  `python reader_bats_bottle.py` 

In [33]:
# System setup and user definitions
from matplotlib import pyplot as plt
import datetime as dt
import numpy as np
import xarray as xr

filedir = './GOTM_INPUT_CLIM/'
varlist = ['Temp', 'Sal1','NO3','NO2','Chl','PO41','O2','CO2', 'Alk','Si1','POC','PON','POP','DOC','DON','DOP']
# AC 15.09.2021 : Paolo, please check the units.
units= {'Temp': '°C',
        'Sal1': '-',
        'NO3' : 'mmolN/m3',
        'NO2' : 'mmolN/m3',
        'Chl' : 'mg/m3',
        'PO41': 'mmolP/m3',
        'O2'  : 'mmolO2/m3',
        'CO2' : 'mgC/m3',
        'Alk' : 'mmol/m3',
        'Si1' : 'mmolSi/m3',
        'POC' : 'mg C/m3',
        'PON' : 'mmolN/m3',
        'POP' : 'mmolP/m3',
        'DOC' : 'mg C/m3',
        'DON' : 'mmolN/m3',
        'DOP' : 'mmolP/m3'}

In [34]:
# AC 15.09.2021 This can be removed if O2_clim.txt is generated instead of 'O2(1)_clim.txt', which is a strange filename
import os
os.system("mv '"+ filedir+"O2(1)_clim.txt' "+filedir+"O2_clim.txt")

mv: cannot stat './GOTM_INPUT_CLIM/O2(1)_clim.txt': No such file or directory


256

In [35]:
def gotm_to_xr(var, units,filedir=filedir):
    '''
    Read and load GOTM-friendly files into xarray structure.
    A. Capet 15/09/2021
    
    Parameters
    ----------
    var : str
        name of a variable. Should correspond to a 'var'_clim.txt file in filedir
    filedir : str
        directory for the climatology files
                
    Returns    
    ----------
    An xarray with the variable var
    '''
    
    fn=filedir+var+'_clim.txt'

    with open(fn) as f:
        tcount=0
        while True:
            txt = f.readline()
            if not txt:
                break
            date,nrow,_=txt.split('\t')
            npa=np.empty([int(nrow),2])
            for n in range(int(nrow)):
                d,v = f.readline().split('\t')
                npa[n,:]=[float(d),float(v)]
            xrl=xr.DataArray(npa[:,1],coords={"depth":npa[:,0]}, dims=['depth'])
            xrl=xrl.expand_dims({"time":[np.datetime64(date)]})
            if tcount==0:
                xrlo=xrl.copy()
            else:
                xrlo=xr.concat([xrlo, xrl],dim="time")
            tcount+=1

    xrlo.attrs["units"] = units[var]
    xrd=xr.Dataset({var:xrlo})
    
    return(xrd)
    
def gotm_clim_load(varlist,filedir,units):
    '''
    Read and load GOTM-friendly files into xarray structure.
    A. Capet 15/09/2021
    
    Parameters
    ----------
    varlist : list of str
        list of variable names. Each should correspond to a 'var'_clim.txt file in filedir
    filedir : str
        directory for the climatology files
    units : dict
        associate units (str) to each var of varlist
                
    Returns    
    ----------
    An xarray dataset with all variables
    '''
    xs = [gotm_to_xr(v,units,filedir) for v in varlist]
    
    xs = xr.merge(xs)
    return(xs)

In [36]:
xrN=gotm_clim_load(varlist,filedir,units)

xrN

<xarray.Dataset>
Dimensions:  (depth: 39, time: 84)
Coordinates:
  * depth    (depth) float64 -1.098e+04 -4.2e+03 -4e+03 ... -20.0 -10.0 -0.0
  * time     (time) datetime64[ns] 2014-01-16T12:00:00 ... 2020-12-16T12:00:00
Data variables: (12/16)
    Temp     (time, depth) float64 2.211 2.211 2.218 2.237 ... 22.77 22.78 22.63
    Sal1     (time, depth) float64 34.88 34.88 34.89 34.89 ... 36.63 36.62 36.65
    NO3      (time, depth) float64 20.43 20.43 20.16 ... 0.005213 0.005214
    NO2      (time, depth) float64 0.01047 0.01047 0.01047 ... 0.005213 0.00782
    Chl      (time, depth) float64 0.0 nan nan nan ... 0.1272 0.1272 0.1116
    PO41     (time, depth) float64 1.361 1.361 1.329 ... 0.005213 0.005214
    ...       ...
    POC      (time, depth) float64 0.0 nan nan nan ... 23.79 23.51 23.65 23.0
    PON      (time, depth) float64 0.0 nan nan nan ... 0.3083 0.3381 0.2886
    POP      (time, depth) float64 0.0 nan nan ... 0.0002354 0.0002354 0.0003364
    DOC      (time, depth) float64 537.4 nan nan nan ... 804.6 818.6 823.4 841.6
    DON      (time, depth) float64 1.71 nan nan nan ... 3.642 4.045 3.911 3.648
    DOP      (time, depth) float64 0.3945 nan nan ... 0.08012 0.07125 0.07178

In [ ]:
###########
ncolplot=3
ylim =-200
###########

nrowplot=int(np.ceil(len(varlist)/ncolplot))

f=plt.figure(figsize=(ncolplot*5,nrowplot*5))
for i,v in enumerate(varlist) : 
    ax=f.add_subplot(nrowplot,ncolplot,i+1)
    xrN[v].plot(x='time',y='depth', ylim=[-200,0], xlim =[xrN.time.values[0],xrN.time.values[24]], ax=ax, cmap='YlGnBu_r')
    ax.set_title(v, fontsize=15)
f.tight_layout()

plt.savefig(filedir+'BatsClims.png', facecolor='white')

In [ ]:
# Some examples
_=xrN.NO3.plot.contourf(x='time',y='depth', ylim=[-300,0], vmax=2.5, levels=20, cmap='YlGnBu_r')

In [ ]:
# Some examples
_=xrN.NO3.plot(x='time',y='depth', ylim=[-2000,0], vmax=15, cmap='YlGnBu_r')

In [ ]:
# Netcdf export
xrN.to_netcdf(filedir+'BATS_CLIM.nc')